In [1]:
import h5pyd
from rex import Resource
from rex import ResourceX
import os
import pandas as pd

In [2]:
print(list(h5pyd.Folder('/nrel/wtk/')))

['wtk', 'NOW-WAKES_Mid_Atlantic', 'alaska', 'bangladesh', 'canada-5min', 'canada', 'canada_bc', 'central_asia', 'conus-5min', 'conus', 'great_lakes-5min', 'great_lakes', 'gulf_of_mexico', 'hawaii-5min', 'hawaii', 'india', 'kazakhstan', 'led', 'maine', 'mexico-5min', 'mexico', 'mid_atlantic-5min', 'mid_atlantic', 'north_america', 'now23_california', 'nw_pacific-5min', 'nw_pacific', 'offshore_ca-5min', 'offshore_ca', 'philippines', 'pr100', 'seasiawind', 'seasiawind_v2', 'seasiawind_v3', 'south_atlantic', 'sup3rwind', 'vietnam', 'wtk-led']


In [3]:
print(list(h5pyd.Folder('/nrel/wtk/great_lakes-5min/')))

['Great_Lakes_2000.h5', 'Great_Lakes_2001.h5', 'Great_Lakes_2002.h5', 'Great_Lakes_2003.h5', 'Great_Lakes_2004.h5', 'Great_Lakes_2005.h5', 'Great_Lakes_2006.h5', 'Great_Lakes_2007.h5', 'Great_Lakes_2008.h5', 'Great_Lakes_2009.h5', 'Great_Lakes_2010.h5', 'Great_Lakes_2011.h5', 'Great_Lakes_2012.h5', 'Great_Lakes_2013.h5', 'Great_Lakes_2014.h5', 'Great_Lakes_2015.h5', 'Great_Lakes_2016.h5', 'Great_Lakes_2017.h5', 'Great_Lakes_2018.h5', 'Great_Lakes_2019.h5', 'Great_Lakes_2020.h5']


In [34]:
with Resource('/nrel/wtk/great_lakes-5min/Great_Lakes_2001.h5',hsds=True) as res:
    print(res.dsets)
    meta = res.meta
    print(meta)
    print(res.time_index)

with ResourceX('/nrel/wtk/great_lakes-5min/Great_Lakes_2001.h5', hsds=True) as res:
    df = res.get_lat_lon_df('windspeed_500m', (42.024938, -76.253072))
    print(df)

['boundary_layer_height', 'coordinates', 'friction_velocity_2m', 'inversemoninobukhovlength_2m', 'meta', 'precipitation_0m', 'pressure_0m', 'pressure_100m', 'pressure_200m', 'pressure_300m', 'relativehumidity_2m', 'roughness_length', 'skin_temperature', 'surface_heat_flux', 'surface_sea_temperature', 'temperature_100m', 'temperature_10m', 'temperature_120m', 'temperature_140m', 'temperature_160m', 'temperature_180m', 'temperature_200m', 'temperature_20m', 'temperature_220m', 'temperature_240m', 'temperature_260m', 'temperature_280m', 'temperature_2m', 'temperature_300m', 'temperature_400m', 'temperature_40m', 'temperature_500m', 'temperature_60m', 'temperature_80m', 'time_index', 'turbulent_kinetic_energy_100m', 'turbulent_kinetic_energy_10m', 'turbulent_kinetic_energy_120m', 'turbulent_kinetic_energy_140m', 'turbulent_kinetic_energy_160m', 'turbulent_kinetic_energy_180m', 'turbulent_kinetic_energy_200m', 'turbulent_kinetic_energy_20m', 'turbulent_kinetic_energy_220m', 'turbulent_kinet

In [4]:
import pandas as pd
profiler_stations = pd.read_csv('data/profiler_locations.csv',usecols=[0,3,4])
for index, row in profiler_stations.iterrows():
    lon = row['lon [degrees]']
    lat = row['lat [degrees]']
    station_id = row['stid']
    print(index, lat, lon, station_id)

0 42.68044 -73.81624 PROF_ALB2
1 42.75175 -73.81128 PROF_ALBA
2 43.78823 -76.11765 PROF_BELL
3 40.872481 -73.893522 PROF_BRON
4 42.99359 -78.79461 PROF_BUFF
5 44.889 -73.46634 PROF_CHAZ
6 42.02143 -79.62746 PROF_CLYM
7 40.970394 -72.20094 PROF_EHAM
8 43.068747 -76.469993 PROF_JORD
9 42.024938 -76.253072 PROF_OWEG
10 40.734335 -73.815856 PROF_QUEE
11 41.99983 -73.88412 PROF_REDH
12 40.604014 -74.148499 PROF_STAT
13 40.919579 -73.133284 PROF_STON
14 41.133034 -74.085979 PROF_SUFF
15 44.224256 -74.441052 PROF_TUPP
16 40.65025 -73.5054 PROF_WANT
17 43.2601 -77.41238 PROF_WEBS


- Not all the locations exist in the Great_lakes domain. So, first check which stations exist where (great_lakes or mid_atlantic)

In [8]:
# Define the height levels and ensure the output directory exists
height_levels = [10] + list(range(20, 301, 20)) + [400, 500]
profiler_stations = pd.read_csv('data/profiler_locations.csv',usecols=[0,3,4])
for index, row in profiler_stations.iloc[5:].iterrows():
    location_found = False
    try:    # Attempt to process each station and handle any errors that occur
        lon = row['lon [degrees]']
        lat = row['lat [degrees]']
        station_id = row['stid']

        output_dir = f'data/NOW23/{station_id}'
        os.makedirs(output_dir, exist_ok=True)
        
        for year in range(2000, 2020+1):  # Adjust the range for additional years if needed

            target_file = f'{output_dir}/{year}.csv'
            if os.path.exists(target_file):
                print(f'Skipping existing file for station {station_id}')
                continue
            
            dfs = []

            # First file path
            file_path = f'/nrel/wtk/great_lakes-5min/Great_Lakes_{year}.h5'
            
            try:
                with ResourceX(file_path, hsds=True) as res:
                    for height in height_levels:
                        var_name = f'windspeed_{height}m'
                        df = res.get_lat_lon_df(var_name, (lat, lon))
                        df = df.rename(columns={df.columns[0]: f'{height}m'})
                        dfs.append(df)
                        location_found = True
            
            except Exception as e:
                print(f'Error with file {file_path}: {e}')
                
                # Attempt second file path
                alt_file_path = f'/nrel/wtk/mid_atlantic-5min/Mid_Atlantic_{year}.h5'
                try:
                    with ResourceX(alt_file_path, hsds=True) as res:
                        for height in height_levels:
                            var_name = f'windspeed_{height}m'
                            df = res.get_lat_lon_df(var_name, (lat, lon))
                            df = df.rename(columns={df.columns[0]: f'{height}m'})
                            dfs.append(df)
                            location_found = True

                except Exception as e:
                    print(f'Error with file {alt_file_path}: {e}')

            # Concatenate all dataframes along the column axis if any data is present
            if dfs:
                combined_df = pd.concat(dfs, axis=1, join='inner')
                # save the combined dataframe to a CSV file
                combined_df.to_csv(target_file)
                print(f'Processed year {year} for station {station_id}')

            else:
                print(f'No data available for station {station_id} in {year}')

            # Break the year loop if location not found in either file path
            if not location_found:
                print(f'No data found for station {station_id} in {year}. Skipping to next station.')
                break

    except Exception as e:
        print(f'Error processing station {station_id}: {e}')
        continue

Error with file /nrel/wtk/great_lakes-5min/Great_Lakes_2000.h5: Latitude, longitude coordinates ([[ 44.889   -73.46634]]) are outsides of the resource domain: ((39.70535659790039, -94.748046875), (50.25122833251953, -73.61572265625))
Error with file /nrel/wtk/mid_atlantic-5min/Mid_Atlantic_2000.h5: Latitude, longitude coordinates ([[ 44.889   -73.46634]]) are outsides of the resource domain: ((37.27253341674805, -76.42779541015625), (42.21065902709961, -64.79974365234375))
No data available for station PROF_CHAZ in 2000
No data found for station PROF_CHAZ in 2000. Skipping to next station.
Processed year 2000 for station PROF_CLYM
Processed year 2001 for station PROF_CLYM
Processed year 2002 for station PROF_CLYM
Processed year 2003 for station PROF_CLYM
Processed year 2004 for station PROF_CLYM
Processed year 2005 for station PROF_CLYM
Processed year 2006 for station PROF_CLYM
Processed year 2007 for station PROF_CLYM
Processed year 2008 for station PROF_CLYM
Processed year 2009 for s